In [1]:
from models.wrapper import CustomCLIPWrapper
from torchvision.models import resnet50
from transformers import AutoTokenizer, AutoModel
import torch

In [2]:
img_encoder = resnet50(pretrained=True)
img_encoder.fc = torch.nn.Linear(2048, 768)

tokenizer = AutoTokenizer.from_pretrained("johngiorgi/declutr-sci-base")
txt_encoder = AutoModel.from_pretrained("johngiorgi/declutr-sci-base")


In [3]:
model = CustomCLIPWrapper.load_from_checkpoint("./logs/version_0/checkpoints/epoch=31-step=7231.ckpt",image_encoder=img_encoder, text_encoder=txt_encoder, minibatch_size=1)

In [6]:
# model = model.model.to('cpu')

In [4]:
from torchvision import transforms as T
import clip

def fix_img(img):
    return img.convert('RGB') if img.mode != 'RGB' else img

image_transform = T.Compose([
    T.Lambda(fix_img),
    T.RandomResizedCrop(224,
                        scale=(0.75, 1.),
                        ratio=(1., 1.)),
    T.ToTensor(),
    T.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
])

/home/jayr/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [21]:
import torch
from PIL import Image

texts = [
  "bird: warm-blooded egg-laying vertebrates characterized by feathers and forelimbs modified as wings",
  "bird: the flesh of a bird or fowl (wild or domestic) used as food",
  "dame: informal terms for a (young) woman",
  "boo: a cry or noise made to express displeasure or contempt",
  "bird: badminton equipment consisting of a ball of cork or rubber with a crown of feathers",
  # 'bird: m. and f. Any land birds, flying, with straight bill not very strong and generally small size.'
]

with torch.no_grad():
    img_emb = image_transform(Image.open("./data_dir/26270.png")).unsqueeze(0).to('cpu')
    img_enc = model.model.encode_image(img_emb)
    # print(img_enc.size())
    # text_emb= clip.tokenize(texts, truncate=True).to('cpu')
    # text_emb = tokenizer(texts,padding='max_length', truncation=True, return_tensors="pt",max_length=77)['input_ids'].to('cpu')
    text_emb = tokenizer(texts,padding='max_length', truncation=True, return_tensors="pt",max_length=77)
    # print(text_emb)/
    text_enc = model.encode_text(text_emb)
    print(text_enc.size())
    logits_per_image, logits_per_text = model(img_emb,text_emb)
    # probs = logits_per_image.softmax(dim=-1).cpu().numpy()
    probs = logits_per_image.cpu().numpy()
    better = probs.argmax()
    print(better)
    print(probs)
    print(texts[better])

torch.Size([5, 768])
2
[[-5.135251  -6.2063804 -1.5040816 -2.0771506 -9.048351 ]]
dame: informal terms for a (young) woman
